In [1]:
from functools import cache
import re
import numpy as np
from sympy import solve, symbols


with open('inputs/day13.txt') as input:
    sections = ''.join(input.readlines()).split('\n\n')
    machines = []
    for section in sections:
        xa, ya, xb, yb, xt, yt = (int(x) for x in re.findall(r'[0-9]+', section))
        machines.append(((xa, ya), (xb, yb), (xt, yt)))

@cache
def old_cheapest(a, b, t):
    if t[0] < 0 or t[1] < 0:
        return np.inf
    if t[0] == 0 and t[1] == 0:
        return 0
    return min(
        1 + old_cheapest(a, b, (t[0] - b[0], t[1] - b[1])),
        3 + old_cheapest(a, b, (t[0] - a[0], t[1] - a[1]))
    )

sum(old_cheapest(*machine) for machine in machines if np.isfinite(old_cheapest(*machine)))

36571

In [2]:
# want n such that
# t[0] - n*a[0] = k*b[0]
# t[1] - n*a[1] = k*b[1]
# for some integers (n, k)
# solve using sympy lol

def find_n_k(a, b, t):
    n, k = symbols('n, k', integer=True)
    result = solve(
        (t[0] - a[0] * n - b[0] * k, t[1] - a[1] * n - b[1] * k),
        n, k
    )
    if not result:
        return np.inf, np.inf
    return int(result[n]), int(result[k])

def cost(n, k):
    return n * 3 + k

def cheapest(a, b, t):
    # compare between performing a until on the line of b, and
    # performing b until on the line of a
    sol1 = find_n_k(a, b, t)
    c1 = cost(sol1[0], sol1[1])
    sol2 = find_n_k(b, a, t)
    c2 = cost(sol2[1], sol2[0])
    return min(c1, c2)

val = 10000000000000
def transform(a, b, t):
    return a, b, (t[0] + val, t[1] + val)

sum(cheapest(*transform(*machine)) for machine in machines if np.isfinite(cheapest(*transform(*machine))))

85527711500010